In [1]:
from pandas_datareader import data as pdr
import yfinance as yf
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AR

# Práctico 3 - Introducción al Aprendizaje Automático

## Modelos de Precios de Mercado
Queremos crear un modelo que prediga si el precio de una acción va a subir o bajar de acuerdo a información del pasado. Para ello implementaremos algunos modelos derivados de la teoria económica respecto de como valuar una activo con flujo de fondos inciertos.


### Capital Asset Pricing Model (CAPM)

El modelo de CAPM propuesto por William Sharpe a partir del desarrollo de Markowitz sobre carteras eficientes, busca predecir el retorno de un activo en función del comportamiento global del mercado, y de la relación del activo puntual con el mercado. 

Según este modelo, que se puede expresar de la siguiente manera: 

$$
E(r_i) = r_f + \beta  [ E(r_m) - r_f ] 
$$

El retorno esperado del activo i, es igual al retorno libre de riesgo (o tasa libre de riesgo), más el exceso de retorno esperado de mercado (es decir lo que se espera que rinda el mercado por sobre la tasa libre de riesgo) escalado por un coeficiente beta. Este coeficiente beta es el conciente entre la covarianza del activo con el mercado sobre la varianza del mercado, y se interpreta como un multiplicador de riesgo de mercado.

Pueden ver una explicación completa en: https://economipedia.com/definiciones/modelo-valoracion-activos-financieros-capm.html

Adicionalmente, se puede agregar un término independiente como ordenada al origen alfa, que represanta una ganancia extraordinaria no explicada por la relacion con de la acción con el mercado sino por factores extraordinarios o arbitrages en caso de estrategias. De allí el famoso "buscar alfa" siendo una directriz de todos los portafolios managers. 

$$
E(r_i) = \alpha + r_f + \beta  [ E(r_m) - r_f ]
$$

Para realizar este ejercicio, deberán descargar el precio de una acción y del mercado asociado (por ejemplo GOOG y NASDAQ). En ambos casos tomar los retornos y tomar una tasa libre de riesgo. A continuación, deberán probar varias regresiones para comprobar el beta, y examinar si el agregado de una ordenada al origen aporta a los resultados. La idea es que seleccionen varias acciones para modelarlo con diferentes activos. 

Una vez que lo hayan hecho, adionalmente responder las siguientes preguntas:

1. ¿Es fácil encontrar $\alpha$ distinto de cero?

2. ¿Qué interpretación le podemos dar a $\beta$?

3. ¿En qué momento nos gustaría buscar activos con $\beta$ > 1 y $\beta$ < 1? ¿Qué implica tener un $\beta$ < 0?



In [2]:
yf.pdr_override() # <== that's all it takes :-)

# download dataframe
data_1 = pdr.get_data_yahoo("GOOG NDAQ", start="2000-01-01", end="2021-07-31")
data_2 = pdr.get_data_yahoo("KO NYA", start="2000-01-01", end="2021-07-31")

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed


In [3]:
data_2

Adj Close                    Close                     High  \
                   KO           NYA         KO           NYA         KO   
Date                                                                      
2000-01-03  15.690016   6762.109863  28.187500   6762.109863  29.000000   
2000-01-04  15.707405   6543.759766  28.218750   6543.759766  28.406250   
2000-01-05  15.846567   6567.029785  28.468750   6567.029785  28.718750   
2000-01-06  15.863964   6635.439941  28.500000   6635.439941  28.843750   
2000-01-07  16.907639   6792.669922  30.375000   6792.669922  30.375000   
...               ...           ...        ...           ...        ...   
2021-07-26  57.060001  16565.300781  57.060001  16565.300781  57.119999   
2021-07-27  57.259998  16521.000000  57.259998  16521.000000  57.540001   
2021-07-28  56.740002  16573.599609  56.740002  16573.599609  57.160000   
2021-07-29  57.049999  16697.099609  57.049999  16697.099609  57.250000   
2021-07-30  57.029999           NaN  57.029999           NaN  57.250000   

                                Low                     Open                \
                     NYA         KO           NYA         KO           NYA   
Date                                                                         
2000-01-03   6762.109863  27.625000   6762.109863  29.000000   6762.109863   
2000-01-04   6543.759766  27.812500   6543.759766  28.187500   6543.759766   
2000-01-05   6567.029785  28.031250   6567.029785  28.218750   6567.029785   
2000-01-06   6635.439941  28.281250   6635.439941  28.468750   6635.439941   
2000-01-07   6792.669922  28.937500   6792.669922  28.937500   6792.669922   
...                  ...        ...           ...        ...           ...   
2021-07-26  16565.300781  56.560001  16565.300781  56.889999  16565.300781   
2021-07-27  16521.000000  56.919998  16521.000000  57.110001  16521.000000   
2021-07-28  16573.599609  56.630001  16573.599609  56.990002  16573.599609   
2021-07-29  16697.099609  56.860001  16697.099609  57.070000  16697.099609   
2021-07-30           NaN  56.900002           NaN  57.150002           NaN   

              Volume       
                  KO  NYA  
Date                       
2000-01-03  10997000  0.0  
2000-01-04   7308000  0.0  
2000-01-05   9457400  0.0  
2000-01-06   7129200  0.0  
2000-01-07  11474000  0.0  
...              ...  ...  
2021-07-26   8681100  0.0  
2021-07-27  12794400  0.0  
2021-07-28   9858000  0.0  
2021-07-29   9599100  0.0  
2021-07-30  11727000  NaN  

[5429 rows x 12 columns]

In [4]:
data_2.isna().sum()

Adj Close  KO       0
           NYA    334
Close      KO       0
           NYA    334
High       KO       0
           NYA    334
Low        KO       0
           NYA    334
Open       KO       0
           NYA    334
Volume     KO       0
           NYA    334
dtype: int64

In [5]:
data_respaldo = data_2
data_2.dropna(inplace=True)
data_2

Adj Close                    Close                     High  \
                   KO           NYA         KO           NYA         KO   
Date                                                                      
2000-01-03  15.690016   6762.109863  28.187500   6762.109863  29.000000   
2000-01-04  15.707405   6543.759766  28.218750   6543.759766  28.406250   
2000-01-05  15.846567   6567.029785  28.468750   6567.029785  28.718750   
2000-01-06  15.863964   6635.439941  28.500000   6635.439941  28.843750   
2000-01-07  16.907639   6792.669922  30.375000   6792.669922  30.375000   
...               ...           ...        ...           ...        ...   
2021-07-23  57.009998  16552.400391  57.009998  16552.400391  57.330002   
2021-07-26  57.060001  16565.300781  57.060001  16565.300781  57.119999   
2021-07-27  57.259998  16521.000000  57.259998  16521.000000  57.540001   
2021-07-28  56.740002  16573.599609  56.740002  16573.599609  57.160000   
2021-07-29  57.049999  16697.099609  57.049999  16697.099609  57.250000   

                                Low                     Open                \
                     NYA         KO           NYA         KO           NYA   
Date                                                                         
2000-01-03   6762.109863  27.625000   6762.109863  29.000000   6762.109863   
2000-01-04   6543.759766  27.812500   6543.759766  28.187500   6543.759766   
2000-01-05   6567.029785  28.031250   6567.029785  28.218750   6567.029785   
2000-01-06   6635.439941  28.281250   6635.439941  28.468750   6635.439941   
2000-01-07   6792.669922  28.937500   6792.669922  28.937500   6792.669922   
...                  ...        ...           ...        ...           ...   
2021-07-23  16552.400391  56.450001  16552.400391  56.590000  16552.400391   
2021-07-26  16565.300781  56.560001  16565.300781  56.889999  16565.300781   
2021-07-27  16521.000000  56.919998  16521.000000  57.110001  16521.000000   
2021-07-28  16573.599609  56.630001  16573.599609  56.990002  16573.599609   
2021-07-29  16697.099609  56.860001  16697.099609  57.070000  16697.099609   

              Volume       
                  KO  NYA  
Date                       
2000-01-03  10997000  0.0  
2000-01-04   7308000  0.0  
2000-01-05   9457400  0.0  
2000-01-06   7129200  0.0  
2000-01-07  11474000  0.0  
...              ...  ...  
2021-07-23  12144000  0.0  
2021-07-26   8681100  0.0  
2021-07-27  12794400  0.0  
2021-07-28   9858000  0.0  
2021-07-29   9599100  0.0  

[5095 rows x 12 columns]

In [6]:
# Retornos
data_2['retornos act'] = data_2['Adj Close', 'KO'].pct_change()*100
data_2['retornos mdo'] = data_2['Adj Close','NYA'].pct_change()*100
data_2

Adj Close                    Close                     High  \
                   KO           NYA         KO           NYA         KO   
Date                                                                      
2000-01-03  15.690016   6762.109863  28.187500   6762.109863  29.000000   
2000-01-04  15.707405   6543.759766  28.218750   6543.759766  28.406250   
2000-01-05  15.846567   6567.029785  28.468750   6567.029785  28.718750   
2000-01-06  15.863964   6635.439941  28.500000   6635.439941  28.843750   
2000-01-07  16.907639   6792.669922  30.375000   6792.669922  30.375000   
...               ...           ...        ...           ...        ...   
2021-07-23  57.009998  16552.400391  57.009998  16552.400391  57.330002   
2021-07-26  57.060001  16565.300781  57.060001  16565.300781  57.119999   
2021-07-27  57.259998  16521.000000  57.259998  16521.000000  57.540001   
2021-07-28  56.740002  16573.599609  56.740002  16573.599609  57.160000   
2021-07-29  57.049999  16697.099609  57.049999  16697.099609  57.250000   

                                Low                     Open                \
                     NYA         KO           NYA         KO           NYA   
Date                                                                         
2000-01-03   6762.109863  27.625000   6762.109863  29.000000   6762.109863   
2000-01-04   6543.759766  27.812500   6543.759766  28.187500   6543.759766   
2000-01-05   6567.029785  28.031250   6567.029785  28.218750   6567.029785   
2000-01-06   6635.439941  28.281250   6635.439941  28.468750   6635.439941   
2000-01-07   6792.669922  28.937500   6792.669922  28.937500   6792.669922   
...                  ...        ...           ...        ...           ...   
2021-07-23  16552.400391  56.450001  16552.400391  56.590000  16552.400391   
2021-07-26  16565.300781  56.560001  16565.300781  56.889999  16565.300781   
2021-07-27  16521.000000  56.919998  16521.000000  57.110001  16521.000000   
2021-07-28  16573.599609  56.630001  16573.599609  56.990002  16573.599609   
2021-07-29  16697.099609  56.860001  16697.099609  57.070000  16697.099609   

              Volume      retornos act retornos mdo  
                  KO  NYA                            
Date                                                 
2000-01-03  10997000  0.0          NaN          NaN  
2000-01-04   7308000  0.0     0.110830    -3.229023  
2000-01-05   9457400  0.0     0.885965     0.355606  
2000-01-06   7129200  0.0     0.109784     1.041721  
2000-01-07  11474000  0.0     6.578901     2.369549  
...              ...  ...          ...          ...  
2021-07-23  12144000  0.0     0.956255     0.586416  
2021-07-26   8681100  0.0     0.087709     0.077937  
2021-07-27  12794400  0.0     0.350503    -0.267431  
2021-07-28   9858000  0.0    -0.908132     0.318380  
2021-07-29   9599100  0.0     0.546347     0.745161  

[5095 rows x 14 columns]

In [7]:
df_retornos = pd.concat([data_2['retornos act'], data_2['retornos mdo']], axis=1, keys=['retornos_act', 'retornos_mdo'])

# Reseteo MultiIndex y renombro columnas
df_retornos = df_retornos.reset_index(level='Date') # Reseteo MultiIndex (Date en este caso y la guardaré como columna)v

df_retornos.dropna(inplace=True) # descarto fechas en que tengo NaN en algún retorno
df_retornos.head()

,Date,retornos_act,retornos_mdo
1,2000-01-04,0.110830,-3.229023
2,2000-01-05,0.885965,0.355606
3,2000-01-06,0.109784,1.041721
4,2000-01-07,6.578901,2.369549
5,2000-01-10,-3.189248,0.673966


$$ E(r_i) = \alpha + r_f + \beta  [ E(r_m) - r_f ]  $$

- E(r_i): rentabilidad esperada del activo
- alpha: ordenada al origine: representa una ganancia extraordinaria
- r_f: rentabilidad activo sin riesgo (riskfree)
- beta: sensibilidad del activo respecto a su benchmark (correlacion entre los retornos del activo y del mercado)
- E(r_m): tasa rentabilidad esperada del mercado en el que cotiza el activo

- r_m - r_f : riesgo asociado al mercado en el que cotiza el activo
- r_i -  r_f : riesgo asociado al activo en concreto

$$ E(r_i)  - r_f = \alpha + \beta [ E(r_m) - r_f ]  $$
$$ retornos del activo = \alpha + \beta (retornos del mercado) $$
$$ y_i = \alpha + \beta (x_i) $$

In [8]:
# TimeSeriesSplit
r_f = 0

x_i = np.array(df_retornos['retornos_mdo'].values - r_f)
y_i = np.array(df_retornos['retornos_act'].values - r_f)


tscv = TimeSeriesSplit(n_splits=3)
# print(tscv) # TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)

for fold, (train_index, test_index) in enumerate(tscv.split(x_i)):
    print("Fold: {}".format(fold))
    print("TRAIN indices:", train_index, "\n", "TEST indices:", test_index)
    print("\n")
    X_train, X_test = x_i[train_index], x_i[test_index]
    y_train, y_test = y_i[train_index], y_i[test_index]
    

Fold: 0
TRAIN indices: [   0    1    2 ... 1272 1273 1274] 
 TEST indices: [1275 1276 1277 ... 2545 2546 2547]


Fold: 1
TRAIN indices: [   0    1    2 ... 2545 2546 2547] 
 TEST indices: [2548 2549 2550 ... 3818 3819 3820]


Fold: 2
TRAIN indices: [   0    1    2 ... 3818 3819 3820] 
 TEST indices: [3821 3822 3823 ... 5091 5092 5093]




In [9]:
# split dependent and independent variable
# X = clean_monthly_returns['^GSPC']
# y = clean_monthly_returns['FB']

# Add a constant to the independent value
# X1 = sm.add_constant(X)

# make regression model 
model = sm.OLS(y_i, x_i)

# fit model and print results
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.290
Model:                            OLS   Adj. R-squared (uncentered):              0.290
Method:                 Least Squares   F-statistic:                              2081.
Date:                Tue, 17 Aug 2021   Prob (F-statistic):                        0.00
Time:                        19:20:12   Log-Likelihood:                         -7978.9
No. Observations:                5094   AIC:                                  1.596e+04
Df Residuals:                    5093   BIC:                                  1.597e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

 <img src="ifinancecocacola.png"/>

El coeficiente del mercado NYSE es de 0,5764 y el beta de coca cola de ifinance es de 0,63

### Modelos Autorregresivos para predecir precios.

Para esta parte del práctico, nos gustaría utilizar precios del pasado para poder estimar cuál va a ser el precio de una acción más adelante. Las regresiones que vemos en la introducción al aprendizaje automático no son las mismas que deben usarse cuando estamos trabajando con precios de mercado, ya que estos datos son series de tiempo. Para trabajar con este tipo de datos, es necesario generar **modelos autorregresivos**.

Si suponemos que $y_t$ es el precio de nuestra acción en el tiempo $t$, podemos generar el siguiente modelo autorregresivo:

$$
y_t = \delta + \phi_1 y_{t-1} + \dots + \phi_p y_{t-p} + \epsilon_t
$$

Existen otras variables que podemos agregar (como estacionariedad o tendencias), pero para este práctico podemos usar solamente el modelo `AutoReg` de la librería StatsModels: https://www.statsmodels.org/stable/examples/notebooks/generated/autoregressions.html.

El **ejercicio** para esta sección es crear una función que reciba 4 parámetros:
- El *ticker* de un activo.
- Una *fecha* a partir de la cual se van a querer predecir los precios del activo.
- Un *número de días* hacia atrás que se van a usar para ajustar el modelo autorregresivo.
- Un *número de días* hacia adelante para los cuales van a querer predecir el precio del activo utilizando el modelo entrenado.

Esta función se debe encargar de buscar los precios del ticker, realizar el ajuste y predecir el precio de la acción hacia delante. Además, debe devolver el [error cuadrático medio](https://es.wikipedia.org/wiki/Error_cuadr%C3%A1tico_medio) de la predicción, comparado con los verdaderos precios del activo.

¿Qué pasa si tratamos de predecir el precio del SPY el día antes del primer cisne negro de 2020 con un modelo que use 15 días hacia atrás?

In [12]:
yf.pdr_override()

start_date = "2000-01-01"
end_date = "2020-12-31"

df_spy = pdr.get_data_yahoo("SPY", start=start_date, end=end_date)
df_spy.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,148.25000,148.25000,143.875000,145.4375,97.506668,8164300
2000-01-04,143.53125,144.06250,139.640625,139.7500,93.693573,8089800
2000-01-05,139.93750,141.53125,137.250000,140.0000,93.861176,12177900
2000-01-06,139.62500,141.50000,137.750000,137.7500,92.352676,6227200
2000-01-07,140.31250,145.75000,140.062500,145.7500,97.716209,8066500


In [18]:
# vamos a utilizar la columna adj close para predecir los precios a partir de dias anteriores.
prices = df_spy[['Adj Close']]

prices.reset_index(level='Date', inplace=True)
prices.head()

,Date,Adj Close
0,2000-01-03,97.506668
1,2000-01-04,93.693573
2,2000-01-05,93.861176
3,2000-01-06,92.352676
4,2000-01-07,97.716209


### Árboles de Decisión para predecir suba o baja.

Venimos trabajando los precios con valores continuos, pero otra forma de verlos podría ser convirtiendolos en, por ejemplo, -1 y 1 para saber si el precio está en suba o en baja. De esta forma obtenemos un problema de clasificación que nos permite mirar los datos desde otra perspectiva.

- Etiqueten los datos con 1 si [pct_change](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html) es mayor a cero y 0, -1 si no.
- Entrenen un árbol de decisión que prediga si el valor de un activo va a subir o bajar.
- Reporten las siguientes métricas sobre el modelo resultante.

    - Accuracy
    - Precision
    - Recall
    - F1
    - matriz de confusión

In [19]:
df_ko = pdr.get_data_yahoo("KO", start="2000-01-01", end="2021-07-31")

[*********************100%***********************]  1 of 1 completed


In [20]:
df_ko.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,29.00000,29.00000,27.62500,28.18750,15.690016,10997000
2000-01-04,28.18750,28.40625,27.81250,28.21875,15.707405,7308000
2000-01-05,28.21875,28.71875,28.03125,28.46875,15.846567,9457400
2000-01-06,28.46875,28.84375,28.28125,28.50000,15.863964,7129200
2000-01-07,28.93750,30.37500,28.93750,30.37500,16.907639,11474000


In [23]:
df_ko['ko_ayer'] = df_ko['Adj Close'].shift(1)
df_ko.head()

,Open,High,Low,Close,Adj Close,Volume,ko_ayer
Date,,,,,,,
2000-01-03,29.00000,29.00000,27.62500,28.18750,15.690016,10997000,NaN
2000-01-04,28.18750,28.40625,27.81250,28.21875,15.707405,7308000,15.690016
2000-01-05,28.21875,28.71875,28.03125,28.46875,15.846567,9457400,15.707405
2000-01-06,28.46875,28.84375,28.28125,28.50000,15.863964,7129200,15.846567
2000-01-07,28.93750,30.37500,28.93750,30.37500,16.907639,11474000,15.863964


In [25]:
df_ko['retorno'] = df_ko['Adj Close'].pct_change()
df_ko

,Open,High,Low,Close,Adj Close,Volume,ko_ayer,retorno
Date,,,,,,,,
2000-01-03,29.000000,29.000000,27.625000,28.187500,15.690016,10997000,NaN,NaN
2000-01-04,28.187500,28.406250,27.812500,28.218750,15.707405,7308000,15.690016,0.001108
2000-01-05,28.218750,28.718750,28.031250,28.468750,15.846567,9457400,15.707405,0.008860
2000-01-06,28.468750,28.843750,28.281250,28.500000,15.863964,7129200,15.846567,0.001098
2000-01-07,28.937500,30.375000,28.937500,30.375000,16.907639,11474000,15.863964,0.065789
...,...,...,...,...,...,...,...,...
2021-07-26,56.889999,57.119999,56.560001,57.060001,57.060001,8681100,57.009998,0.000877
2021-07-27,57.110001,57.540001,56.919998,57.259998,57.259998,12794400,57.060001,0.003505
2021-07-28,56.990002,57.160000,56.630001,56.740002,56.740002,9858000,57.259998,-0.009081


In [ ]:
df_ko['target'] = df_ko['retorno']

El etiquetado de datos no suele ser trivial dado a que depende de la naturaleza de los datos. Posiblemente, el precio de un activo suba un poco pero luego continue su declive ¿Cuánto tiene que subir/bajar un precio para ser considerado "suba"/"baja"?. ¿Tiene sentido tomar "periodos de suba/baja" más grandes para predecir si el comportamiento del activo en el futuro?. ¿Se podría utilizar un enfoque más estadístico para elegir el etiquetado?.

Aplicar GridSearch sobre los parámetros y luego para la mejor configuración encontrada, evaluar sobre el conjunto de entrenamiento y sobre el conjunto de evaluación, reportando:

- Accuracy
- Precision
- Recall
- F1
- matriz de confusión